# Case Study - Freddie Mac Bonds


## 1. Pricing the Callable Bond


### Data

Use the data from the following files.
* `../data/callable_bonds_2025-02-13.xlsx`
* `../data/discount_curve_2025-02-13.xlsx`


The data contains info on the following bonds.

`Callable`
* `FHLMC 4.41 01/28/30` is a callable bond, and it is the primary object of our analysis.


In [26]:
FILE_BOND = 'data/callable_bonds_2025-02-13.xlsx'
FILE_CURVE = 'data/discount_curve_2025-02-13.xlsx'

KEY_CALLABLE = 'FHLMC 4.41 01/28/30'

### Bond Info


In [27]:
import pandas as pd
import numpy as np
import math

info = pd.read_excel(FILE_BOND,sheet_name='info').set_index('info')
info_core = info[[KEY_CALLABLE]]
info_core.style.format('{:.2%}',subset=pd.IndexSlice[["Cpn Rate"], :]).format('{:,.0f}',subset=pd.IndexSlice[["Amount Issued"], :]).format('{:%Y-%m-%d}',subset=pd.IndexSlice[["Date Quoted","Date Issued","Date Matures","Date Next Call","Date of First Possible Call"], :])

,FHLMC 4.41 01/28/30
info,
CUSIP,3134HA4V2
Issuer,FREDDIE MAC
Maturity Type,CALLABLE
Issuer Industry,GOVT AGENCY
Amount Issued,"10,000,000"
Cpn Rate,4.41%
Cpn Freq,2
Date Quoted,2025-02-13
Date Issued,2025-01-28


### Quoted Values


In [28]:
quotes = pd.read_excel(FILE_BOND,sheet_name='quotes').set_index('quotes')
quotes_core = quotes[[KEY_CALLABLE]]
quotes_core.style.format('{:.2f}', subset=pd.IndexSlice[quotes.index[1:], :]).format('{:%Y-%m-%d}', subset=pd.IndexSlice['Date Quoted', :])

,FHLMC 4.41 01/28/30
quotes,
Date Quoted,2025-02-13
TTM,4.96
Clean Price,99.89
Dirty Price,100.09
Accrued Interest,0.20
YTM Call,4.45
YTM Maturity,4.43
Duration,4.50
Modified Duration,4.40


### Discount Curves


In [29]:
discs = pd.read_excel(FILE_CURVE,sheet_name='discount curve').set_index('ttm')
display(discs.head())
display(discs.tail())

,maturity date,spot rate,discount
ttm,,,
0.5,2025-08-13,0.043743,0.978597
1.0,2026-02-13,0.042890,0.958451
1.5,2026-08-13,0.042238,0.939228
2.0,2027-02-13,0.041843,0.920515
2.5,2027-08-13,0.041632,0.902117


,maturity date,spot rate,discount
ttm,,,
28.0,2053-02-13,0.040185,0.328231
28.5,2053-08-13,0.040051,0.322978
29.0,2054-02-13,0.039916,0.317851
29.5,2054-08-13,0.039791,0.312766
30.0,2055-02-13,0.039665,0.307802


### 1.1.

Use the discount curve data to price both the `callable` and `reference` bonds.

Also calculate the price of the `hypothetical` bonds, where we consider a non-callable version of the callable bond with 
* maturity unchanged
* maturity at the call date.


In [30]:
def _pick_reference_key(info, key_callable):
    candidates = [k for k in info.columns if k != key_callable]
    if len(candidates) == 0:
        raise ValueError("No reference bond found in info table.")
    # choose bond with maturity closest to callable maturity
    tgt = pd.Timestamp(info.loc["Date Matures", key_callable])
    mats = pd.Series({k: pd.Timestamp(info.loc["Date Matures", k]) for k in candidates})
    return (mats - tgt).abs().idxmin()

def _coupon_schedule(issue_date, maturity_date, freq):
    step_months = int(round(12 / freq))
    d = pd.Timestamp(issue_date)
    maturity_date = pd.Timestamp(maturity_date)
    dates = []
    while d <= maturity_date:
        dates.append(d)
        d = d + pd.DateOffset(months=step_months)
    if dates[-1] != maturity_date:
        dates.append(maturity_date)
    return pd.DatetimeIndex(dates)

def _interp_discount(ttm, discs):
    x = discs.index.astype(float).values
    y = discs["discount"].astype(float).values
    return float(np.interp(ttm, x, y))

def price_straight_from_curve(bond_key, info, discs, maturity_override=None, par=100.0):
    quote_date = pd.Timestamp(info.loc["Date Quoted", bond_key])
    issue_date = pd.Timestamp(info.loc["Date Issued", bond_key])
    maturity = pd.Timestamp(maturity_override if maturity_override is not None else info.loc["Date Matures", bond_key])

    cpn = float(info.loc["Cpn Rate", bond_key])      # annual coupon rate, decimal
    freq = int(info.loc["Cpn Freq", bond_key])       # payments per year
    coupon_amt = par * cpn / freq

    sched = _coupon_schedule(issue_date, maturity, freq)

    # Previous / next coupon around quote date (for accrued interest)
    prev_cpn = sched[sched <= quote_date].max()
    next_cpn = sched[sched > quote_date].min()
    if pd.isna(prev_cpn):
        prev_cpn = issue_date
    if pd.isna(next_cpn):
        next_cpn = maturity

    accrual_frac = (quote_date - prev_cpn).days / (next_cpn - prev_cpn).days
    accrued = coupon_amt * accrual_frac

    # Future cash flows discounted from curve
    cf_dates = sched[sched > quote_date]
    ttms = np.array([(d - quote_date).days / 365.25 for d in cf_dates], dtype=float)

    cfs = np.full(len(cf_dates), coupon_amt, dtype=float)
    cfs[-1] += par

    dfs = np.array([_interp_discount(t, discs) for t in ttms], dtype=float)
    dirty = float(np.sum(cfs * dfs))
    clean = dirty - accrued

    return {
        "bond": bond_key,
        "maturity_used": maturity.date(),
        "dirty_model": dirty,
        "clean_model": clean,
        "accrued_model": accrued
    }

# Identify reference bond if not predefined
if "KEY_REFERENCE" not in globals():
    KEY_REFERENCE = _pick_reference_key(info, KEY_CALLABLE)

call_date = pd.Timestamp(info.loc["Date Next Call", KEY_CALLABLE])

rows = []
# 1) Callable bond (priced as straight bond to maturity from discount curve)
rows.append({"case": "callable (straight to maturity)", **price_straight_from_curve(KEY_CALLABLE, info, discs)})

# 2) Reference bond
rows.append({"case": "reference", **price_straight_from_curve(KEY_REFERENCE, info, discs)})

# 3) Hypothetical: non-callable version of callable, maturity unchanged
rows.append({"case": "hypo callable non-callable, same maturity", **price_straight_from_curve(KEY_CALLABLE, info, discs)})

# 4) Hypothetical: non-callable version of callable, maturity set to call date
rows.append({"case": "hypo callable non-callable, matures at call date", **price_straight_from_curve(KEY_CALLABLE, info, discs, maturity_override=call_date)})

out = pd.DataFrame(rows).set_index("case")

# Add market comparables when available
for idx in out.index:
    b = out.loc[idx, "bond"]
    if b in quotes.columns:
        out.loc[idx, "dirty_mkt"] = float(quotes.loc["Dirty Price", b])
        out.loc[idx, "clean_mkt"] = float(quotes.loc["Clean Price", b])

out[["bond", "maturity_used", "dirty_model", "clean_model", "accrued_model", "dirty_mkt", "clean_mkt"]]

,bond,maturity_used,dirty_model,clean_model,accrued_model,dirty_mkt,clean_mkt
case,,,,,,,
callable (straight to maturity),FHLMC 4.41 01/28/30,2030-01-28,101.410165,101.215248,0.194917,100.089000,99.8930
reference,FHLMC 1 1/4 01/29/30,2030-01-29,87.243000,87.191205,0.051796,85.161583,85.1095
"hypo callable non-callable, same maturity",FHLMC 4.41 01/28/30,2030-01-28,101.410165,101.215248,0.194917,100.089000,99.8930
"hypo callable non-callable, matures at call date",FHLMC 4.41 01/28/30,2028-01-28,100.903803,100.708886,0.194917,100.089000,99.8930


### 1.2.

Calculate the forward price of the `hypothetical` bond as of the date that the `callable` bond can be exercised.

Use the information from the discount curve (and associated forward curve) to calculate this forward price.


In [31]:
# 1.2 Forward price of the hypothetical (non-callable) bond at call date

# Underlying = hypothetical non-callable version of KEY_CALLABLE
bond_key = KEY_CALLABLE

quote_date = pd.Timestamp(info.loc["Date Quoted", bond_key])
call_date = pd.Timestamp(info.loc["Date Next Call", bond_key])
maturity = pd.Timestamp(info.loc["Date Matures", bond_key])
issue_date = pd.Timestamp(info.loc["Date Issued", bond_key])

cpn = float(info.loc["Cpn Rate", bond_key])
freq = int(info.loc["Cpn Freq", bond_key])
par = 100.0
coupon_amt = par * cpn / freq

# Full coupon schedule
sched = _coupon_schedule(issue_date, maturity, freq)

# Cash flows strictly after exercise date (forward delivers bond at call date)
cf_dates_fwd = sched[sched > call_date]
cfs_fwd = np.full(len(cf_dates_fwd), coupon_amt, dtype=float)
cfs_fwd[-1] += par  # add principal at final maturity

# Discount factors from t=0 curve
ttm_call = (call_date - quote_date).days / 365.25
df_call = _interp_discount(ttm_call, discs)

ttms_fwd = np.array([(d - quote_date).days / 365.25 for d in cf_dates_fwd], dtype=float)
dfs_fwd = np.array([_interp_discount(t, discs) for t in ttms_fwd], dtype=float)

# Forward dirty price at call date:
# F(0,T_call) = PV0(cashflows after T_call) / D(0,T_call)
forward_dirty = float(np.sum(cfs_fwd * dfs_fwd) / df_call)

# Also report a cc zero rate to call date from discount curve (useful for 1.3)
r_cc_to_call = -np.log(df_call) / ttm_call

pd.DataFrame(
    {
        "quote_date": [quote_date.date()],
        "call_date": [call_date.date()],
        "T_call_years": [ttm_call],
        "D(0,T_call)": [df_call],
        "cc_zero_to_call": [r_cc_to_call],
        "forward_dirty_at_call": [forward_dirty],
    }
).T.rename(columns={0: "value"})

,value
quote_date,2025-02-13
call_date,2028-01-28
T_call_years,2.954141
"D(0,T_call)",0.885738
cc_zero_to_call,0.041073
forward_dirty_at_call,100.571684


### 1.3.

The provided implied vol corresponds to the implied vol of the **rate**. Specifically,
* the forward rate corresponding to the time of expiration.
* continuously compounded.

Use the duration approximation to get the approximate implied vol corresponding to the forward price.

$$\sigma_{\text{bond fwd price}} \approx D \times \sigma_{\text{fwd rate}}\times f(T_1)$$

where $f(T_1)$ is the continuously-compounded (instantaneous) forward rate at time $T_1$.
* If you're struggling with the forward rate calc, just usse the provided spot rate at $T_1$; it will be a close approximation in this example.
* In this approximation, use the quoted duration from the table. (Yes, this is a bit circular, but we don't want to get bogged down with a duration calculation at this point.)

Report the implied vol of the bond's forward price.


In [32]:
# 1.3 Approximate implied vol of the bond forward price
# Uses: discs, quotes, info, KEY_CALLABLE, and (optionally) ttm_call from 1.2

bond_key = KEY_CALLABLE

# Time to exercise (reuse from 1.2 if available)
if "ttm_call" not in globals():
    quote_date = pd.Timestamp(info.loc["Date Quoted", bond_key])
    call_date = pd.Timestamp(info.loc["Date Next Call", bond_key])
    ttm_call = (call_date - quote_date).days / 365.25

# Quoted duration from table (per instructions)
D_quoted = float(quotes.loc["Duration", bond_key])

# Implied vol of forward RATE from table
sigma_rate_raw = float(quotes.loc["Implied Vol", bond_key])
sigma_rate = sigma_rate_raw / 100.0 if sigma_rate_raw > 1 else sigma_rate_raw  # handle 23.88 vs 0.2388

# Instantaneous cc forward rate f(T1) from discount curve:
# f(t) = - d/dt ln D(0,t), approximated by finite differences on curve grid.
x = discs.index.astype(float).values
d = discs["discount"].astype(float).values

def inst_fwd_cc(t, x, d):
    # local slope of ln D around t using neighboring curve points
    lnD = np.log(d)
    i = np.searchsorted(x, t)
    if i <= 0:
        slope = (lnD[1] - lnD[0]) / (x[1] - x[0])      # forward diff
    elif i >= len(x):
        slope = (lnD[-1] - lnD[-2]) / (x[-1] - x[-2])  # backward diff
    else:
        slope = (lnD[i] - lnD[i-1]) / (x[i] - x[i-1])  # local secant
    return -slope

f_T1 = inst_fwd_cc(ttm_call, x, d)

# Also compute spot-rate-at-T1 fallback (as allowed in prompt)
D_T1 = float(np.interp(ttm_call, x, d))
z_T1 = -np.log(D_T1) / ttm_call  # cc spot to T1

# Duration approximation:
# sigma_bond_fwd_price ≈ D * sigma_fwd_rate * f(T1)
sigma_bond_from_fwd = D_quoted * sigma_rate * f_T1
sigma_bond_from_spot = D_quoted * sigma_rate * z_T1  # fallback approximation

pd.DataFrame({
    "Duration_used": [D_quoted],
    "sigma_fwd_rate_raw_quote": [sigma_rate_raw],
    "sigma_fwd_rate_decimal": [sigma_rate],
    "T1_years": [ttm_call],
    "f_T1_cc_instantaneous": [f_T1],
    "z_T1_cc_spot_fallback": [z_T1],
    "sigma_bond_fwd_price_from_f(T1)": [sigma_bond_from_fwd],
    "sigma_bond_fwd_price_from_spot(T1)": [sigma_bond_from_spot],
}).T.rename(columns={0: "value"})

,value
Duration_used,4.496738
sigma_fwd_rate_raw_quote,23.879829
sigma_fwd_rate_decimal,0.238798
T1_years,2.954141
f_T1_cc_instantaneous,0.040385
z_T1_cc_spot_fallback,0.041073
sigma_bond_fwd_price_from_f(T1),0.043366
sigma_bond_fwd_price_from_spot(T1),0.044104


### 1.4.

For the `callable` bond, report Black's value of the embedded call option.
* Use this to report the value of the `callable` bond.
* How does it compare to the actual market price?

For the calculation of the option, use...
* the quoted `Implied Vol` calculated above.
* forward price of the `hypothetical` bond calculated above.
* provided discount factor

#### Simplifications
Note that in this calculation we are making a few simplifications.
* We are simplifying that the `callable` bond is European exercise with an exercise date as reported in `Date Next Call` above. 
* In reality, it is Bermudan, with quarterly exercise dates after the first exercise date.
* The time-to-exercise is not a round number, but you only have discount factors at rounded time-to-maturities. Just use the closest discount factor.


In [33]:
# 1.4 Black value of embedded call option and callable bond value
# (builds on 1.1/1.2; uses forward_dirty, df_call, ttm_call if already computed)

def _norm_cdf(x):
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

# --- Reuse / rebuild key inputs from earlier parts ---
bond_key = KEY_CALLABLE
quote_date = pd.Timestamp(info.loc["Date Quoted", bond_key])
call_date = pd.Timestamp(info.loc["Date Next Call", bond_key])

# If 1.2 vars are not in memory, recreate them
if "ttm_call" not in globals():
    ttm_call = (call_date - quote_date).days / 365.25

if "df_call" not in globals():
    df_call = _interp_discount(ttm_call, discs)

if "forward_dirty" not in globals():
    # Forward dirty price of hypothetical (non-callable) bond at call date
    maturity = pd.Timestamp(info.loc["Date Matures", bond_key])
    issue_date = pd.Timestamp(info.loc["Date Issued", bond_key])
    cpn = float(info.loc["Cpn Rate", bond_key])
    freq = int(info.loc["Cpn Freq", bond_key])
    par = 100.0
    coupon_amt = par * cpn / freq

    sched = _coupon_schedule(issue_date, maturity, freq)
    cf_dates_fwd = sched[sched > call_date]  # cashflows after delivery date
    cfs_fwd = np.full(len(cf_dates_fwd), coupon_amt, dtype=float)
    cfs_fwd[-1] += par

    ttms_fwd = np.array([(d - quote_date).days / 365.25 for d in cf_dates_fwd], dtype=float)
    dfs_fwd = np.array([_interp_discount(t, discs) for t in ttms_fwd], dtype=float)

    forward_dirty = float(np.sum(cfs_fwd * dfs_fwd) / df_call)

# Vol for Black should be vol of bond forward price (from 1.3 approximation)
if "sigma_bond_fwd" not in globals():
    duration = float(quotes.loc["Duration", bond_key])                     # quoted duration
    sigma_fwd_rate = float(quotes.loc["Implied Vol", bond_key]) / 100.0    # quoted rate vol (decimal)
    f_T1 = float(np.interp(ttm_call,
                           discs.index.astype(float).values,
                           discs["spot rate"].astype(float).values))        # allowed approximation
    sigma_bond_fwd = duration * sigma_fwd_rate * f_T1

# --- Black-76 call on forward price ---
K = float(info.loc["Strike", bond_key])   # call strike (price terms)
T = float(ttm_call)
sig = float(sigma_bond_fwd)

if sig <= 0 or T <= 0:
    call_pv = df_call * max(forward_dirty - K, 0.0)
    d1 = np.nan
    d2 = np.nan
else:
    volT = sig * math.sqrt(T)
    d1 = (math.log(forward_dirty / K) + 0.5 * sig * sig * T) / volT
    d2 = d1 - volT
    call_pv = df_call * (forward_dirty * _norm_cdf(d1) - K * _norm_cdf(d2))

# Callable bond value = straight (hypothetical non-callable) - embedded call option
if "out" in globals() and "dirty_model" in out.columns:
    straight_dirty = float(out.loc["hypo callable non-callable, same maturity", "dirty_model"])
    accrued = float(out.loc["hypo callable non-callable, same maturity", "accrued_model"])
else:
    straight = price_straight_from_curve(bond_key, info, discs)
    straight_dirty = float(straight["dirty_model"])
    accrued = float(straight["accrued_model"])

callable_dirty_model = straight_dirty - call_pv
callable_clean_model = callable_dirty_model - accrued

mkt_dirty = float(quotes.loc["Dirty Price", bond_key])
mkt_clean = float(quotes.loc["Clean Price", bond_key])

res_14 = pd.DataFrame({
    "T_call_years": [T],
    "D(0,T_call)": [df_call],
    "Forward_dirty_hypo": [forward_dirty],
    "K": [K],
    "sigma_bond_fwd": [sig],
    "d1": [d1],
    "d2": [d2],
    "Embedded_call_value_PV": [call_pv],
    "Straight_dirty_model": [straight_dirty],
    "Callable_dirty_model": [callable_dirty_model],
    "Callable_clean_model": [callable_clean_model],
    "Callable_dirty_mkt": [mkt_dirty],
    "Callable_clean_mkt": [mkt_clean],
    "Diff_dirty_model_minus_mkt": [callable_dirty_model - mkt_dirty],
    "Diff_clean_model_minus_mkt": [callable_clean_model - mkt_clean],
}, index=[bond_key])

res_14.T

,FHLMC 4.41 01/28/30
T_call_years,2.954141
"D(0,T_call)",0.885738
Forward_dirty_hypo,100.571684
K,100.000000
sigma_bond_fwd,0.044569
d1,0.112719
d2,0.036116
Embedded_call_value_PV,2.974590
Straight_dirty_model,101.410165
Callable_dirty_model,98.435575


### 1.5.

Calculate the YTM of the callable bond, assuming that...
* it can never be called. (This is the `hypothetical` bond we analyzed above.)
* it will certainly be called.

How do these compare to the quoted YTM Called and YTM Maturity in the table?


In [38]:
# 1.5 YTM of callable bond under two assumptions:
# (i) never called (to maturity), (ii) certainly called (to next call date)

bond_key = KEY_CALLABLE
quote_date = pd.Timestamp(info.loc["Date Quoted", bond_key])
issue_date = pd.Timestamp(info.loc["Date Issued", bond_key])

maturity_date = pd.Timestamp(info.loc["Date Matures", bond_key])
call_date = pd.Timestamp(info.loc["Date Next Call", bond_key])

cpn = float(info.loc["Cpn Rate", bond_key])           # decimal, e.g. 0.0441
freq = int(info.loc["Cpn Freq", bond_key])            # e.g. 2
par = 100.0
coupon_amt = par * cpn / freq

# market price input (use dirty price)
dirty_mkt = float(quotes.loc["Dirty Price", bond_key])

def _future_cfs(bond_key, maturity_override=None):
    maturity_used = pd.Timestamp(maturity_override if maturity_override is not None else info.loc["Date Matures", bond_key])
    sched = _coupon_schedule(issue_date, maturity_used, freq)
    cf_dates = sched[sched > quote_date]
    cfs = np.full(len(cf_dates), coupon_amt, dtype=float)
    cfs[-1] += par
    ttms = np.array([(d - quote_date).days / 365.25 for d in cf_dates], dtype=float)
    return ttms, cfs

def _price_from_ytm(y, ttms, cfs, freq):
    # Street-style periodic compounding with potentially non-integer periods
    return float(np.sum(cfs / (1 + y / freq) ** (freq * ttms)))

def _solve_ytm_bisect(target_dirty, ttms, cfs, freq, lo=-0.20, hi=1.00, tol=1e-12, max_iter=300):
    def f(y): return _price_from_ytm(y, ttms, cfs, freq) - target_dirty

    flo, fhi = f(lo), f(hi)
    # expand bracket if needed
    k = 0
    while flo * fhi > 0 and k < 50:
        lo -= 0.10
        hi += 0.20
        flo, fhi = f(lo), f(hi)
        k += 1
    if flo * fhi > 0:
        raise RuntimeError("Could not bracket YTM root.")

    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        fm = f(mid)
        if abs(fm) < tol:
            return mid
        if flo * fm <= 0:
            hi, fhi = mid, fm
        else:
            lo, flo = mid, fm
    return 0.5 * (lo + hi)

def _norm_quote_yield(x):
    # Some sheets store yields as 4.45 (percent points), others 0.0445
    x = float(x)
    return x / 100.0 if x > 1.0 else x

# Solve model-implied YTMs from the same market dirty price
ttm_mat, cfs_mat = _future_cfs(bond_key, maturity_override=maturity_date)  # never called
ttm_call, cfs_call = _future_cfs(bond_key, maturity_override=call_date)     # certainly called

ytm_never_called = _solve_ytm_bisect(dirty_mkt, ttm_mat, cfs_mat, freq)
ytm_certainly_called = _solve_ytm_bisect(dirty_mkt, ttm_call, cfs_call, freq)

# Quoted comparables
ytm_call_quoted = _norm_quote_yield(quotes.loc["YTM Call", bond_key])
ytm_mat_quoted  = _norm_quote_yield(quotes.loc["YTM Maturity", bond_key])

out_15 = pd.DataFrame(
    [
        {
            "assumption": "never called (to maturity)",
            "ytm_model": ytm_never_called,
            "ytm_quoted": ytm_mat_quoted,
            "diff_bps (model-quoted)": 1e4 * (ytm_never_called - ytm_mat_quoted),
        },
        {
            "assumption": "certainly called (to next call date)",
            "ytm_model": ytm_certainly_called,
            "ytm_quoted": ytm_call_quoted,
            "diff_bps (model-quoted)": 1e4 * (ytm_certainly_called - ytm_call_quoted),
        },
    ]
)

out_15.style.format(
    {
        "ytm_model": "{:.4%}",
        "ytm_quoted": "{:.4%}",
        "diff_bps (model-quoted)": "{:+.2f}",
    }
)

,assumption,ytm_model,ytm_quoted,diff_bps (model-quoted)
0,never called (to maturity),4.4343%,4.4338%,+0.05
1,certainly called (to next call date),4.4508%,4.4483%,+0.25


In the table above, the calculated and quoted YTMs align very closely; however, our model produces slightly higher values for both YTM-to-maturity and YTM-to-next-call, with differences of less than 1 basis point.

### 1.6.

Calculate the duration of...
* the `hypothetical` bond
* the `callable` bond

How do these compare to the quoted duration in the table?

For the callable bond, calculate duration numerically by modifying the spot rates up and down by 1bp and seeing how it changes the valuation of parts `1.1`-`1.3`.


### 1.7.

Calculate the OAS of the `callable` bond.

How does it compare to the quoted OAS?

Recall that the OAS is the parallel shift in the spot curve needed to align the modeled value to the market quote.


### 1.8. Optional OTM Callables


There are a few other Freddie Mac callables that may be of interest.
* `FHLMC 0.97 01/28/28`
* `FHLMC 1.25 01/29/30`

Though these are technically callable, they are far out of the money (OTM). 
* Expiring in 3 months, though code below changes it to 6 monhts, to match coupon.
* These don't have interesting convexity due to being so far OTM.


In [34]:
# KEY_CALLABLE = 'FHLMC 1 1/4 01/29/30'
# KEY_CALLABLE = 'FHLMC 0.97 01/28/28'

In [35]:
info.style.format('{:.2%}',subset=pd.IndexSlice[["Cpn Rate"], :]).format('{:,.0f}',subset=pd.IndexSlice[["Amount Issued"], :]).format('{:%Y-%m-%d}',subset=pd.IndexSlice[["Date Quoted","Date Issued","Date Matures","Date Next Call","Date of First Possible Call"], :])

,FHLMC 0.97 01/28/28,FHLMC 1 1/4 01/29/30,FHLMC 4.41 01/28/30
info,,,
CUSIP,3134GW5F9,3134GWGK6,3134HA4V2
Issuer,FREDDIE MAC,FREDDIE MAC,FREDDIE MAC
Maturity Type,CALLABLE,CALLABLE,CALLABLE
Issuer Industry,GOVT AGENCY,GOVT AGENCY,GOVT AGENCY
Amount Issued,"30,000,000","25,000,000","10,000,000"
Cpn Rate,0.97%,1.25%,4.41%
Cpn Freq,2,2,2
Date Quoted,2025-02-13,2025-02-13,2025-02-13
Date Issued,2020-10-28,2020-07-29,2025-01-28


In [36]:
quotes.style.format('{:.2f}', subset=pd.IndexSlice[quotes.index[1:], :]).format('{:%Y-%m-%d}', subset=pd.IndexSlice['Date Quoted', :])

,FHLMC 0.97 01/28/28,FHLMC 1 1/4 01/29/30,FHLMC 4.41 01/28/30
quotes,,,
Date Quoted,2025-02-13,2025-02-13,2025-02-13
TTM,2.95,4.96,4.96
Clean Price,90.14,85.11,99.89
Dirty Price,90.19,85.16,100.09
Accrued Interest,0.04,0.05,0.20
YTM Call,54.24,85.40,4.45
YTM Maturity,4.57,4.65,4.43
Duration,2.92,4.81,4.50
Modified Duration,2.85,4.70,4.40


### 1.9. ATM with 1-yr expiry

Try this alternate file `2025-02-18` for a recently-issued bond of size $1bn with a one-year expiration.
* Easier to see the negative convexity.
* Large size, recency should be more liquid.


In [37]:
# FILE_BOND = '../data/callable_bonds_2025-02-18.xlsx'
# FILE_CURVE = '../data/discount_curve_2025-02-18.xlsx'
# KEY_CALLABLE = 'FHLMC 4.55 02/11/28'